## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-28-01-00-00 +0000,wsj,What Is a City When Its Wealthiest Leave?,https://www.wsj.com/real-estate/miami-property...
1,2026-02-28-00-57-50 +0000,nypost,Gavin Newsom touts ‘most ambitious water plan’...,https://nypost.com/2026/02/27/us-news/gavin-ne...
2,2026-02-28-00-57-36 +0000,missionlocal,"S.F. court strike ends on Day 2, services to r...",https://missionlocal.org/2026/02/sf-court-cler...
3,2026-02-28-00-47-30 +0000,nyt,Taiwan Arms Sale Approved by Congress Is Delay...,https://www.nytimes.com/2026/02/27/us/politics...
4,2026-02-28-00-41-54 +0000,latimes,"Power, politics and a $2.8-billion exit: How P...",https://www.latimes.com/entertainment-arts/bus...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2595/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,35
163,ai,19
38,warner,18
178,iran,17
238,war,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
56,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...,168
175,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...,117
11,2026-02-28-00-22-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...,100
179,2026-02-27-18-03-16 +0000,nypost,Forcing Bill Clinton to testify will backfire ...,https://nypost.com/2026/02/27/us-news/forcing-...,94
90,2026-02-27-22-05-20 +0000,nypost,Trump orders federal agencies to stop using An...,https://nypost.com/2026/02/27/business/trump-o...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
56,168,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...
100,88,2026-02-27-21-50-27 +0000,nypost,Warner Bros. Discovery CEO David Zaslav calls ...,https://nypost.com/2026/02/27/business/warner-...
337,70,2026-02-27-03-31-25 +0000,nypost,JD Vance says ‘no chance’ a US attack on Iran ...,https://nypost.com/2026/02/26/us-news/jd-vance...
142,46,2026-02-27-19-54-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
11,45,2026-02-28-00-22-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
78,44,2026-02-27-22-31-40 +0000,cbc,Minister leaves door open to tabling new polic...,https://www.cbc.ca/news/politics/anandasangare...
305,42,2026-02-27-09-47-19 +0000,bbc,Green Party wins Gorton and Denton by-election...,https://www.bbc.com/news/articles/cr453rvy6kvo...
48,39,2026-02-27-23-24-00 +0000,wsj,Kristi Noem for months failed to respond to an...,https://www.wsj.com/politics/national-security...
144,39,2026-02-27-19-52-17 +0000,startribune,"Anderson: Growing up in south Minneapolis, fis...",https://www.startribune.com/dennis-anderson-la...
258,36,2026-02-27-12-02-50 +0000,nypost,Huckabee tells US embassy staff in Israel to l...,https://nypost.com/2026/02/27/us-news/mike-huc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
